In [11]:
!pip install openai gradio nltk torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.1 MB/s eta 0:00:00


In [2]:
import json
import torch
import gradio as gr
from openai import OpenAI

In [19]:
jsonl_data = {}
with open('new_intents8.json', 'r') as file:
    jsonl_data = json.load(file)
data = jsonl_data['intents']
formatted_data = []

for intent in data:
    formatted_data.append({
        "messages": [{"role": "system", "content": "ChatBot for the First Aid and recommendation."},
                     {"role": "system", "content": "Write answers for only a First Aid related question and do not answer other questions."},
                     {"role": "system", "content": "After every response recommend a helpline number or hospital for that particular field. All users are from Canada, Thunder Bay."},
            {"role": "user", "content": intent['patterns'][0]},
            {"role": "user", "content": intent['patterns'][1]},
            {"role": "user", "content": intent['patterns'][2]},
            {"role": "user", "content": intent['patterns'][3]},
            {"role": "assistant", "content": intent['responses'][0]}
        ]
    })

with open('formatted_data.jsonl', 'w') as f:
    for item in formatted_data:
        f.write(json.dumps(item) + '\n')

print("Required data is saved in this file 'formatted_data.jsonl'")


Required data is saved in this file 'formatted_data.jsonl'


In [20]:
client = OpenAI(api_key="Your API key")
with open("formatted_data.jsonl", "r") as f:
    data = [json.loads(line) for line in f]



In [21]:
file = client.files.create(
  file=open("formatted_data.jsonl", "rb"),
  purpose="fine-tune"
)

client.fine_tuning.jobs.create(
  training_file=file.id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-lyXcptTi8KFAjdMAeEiuIiE3', created_at=1712085355, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-zuryypGfetQ6X0k1X2M2uw47', result_files=[], status='validating_files', trained_tokens=None, training_file='file-UQTKBZG6KKPkE4afW6MvCY5m', validation_file=None, user_provided_suffix=None)

In [30]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-lyXcptTi8KFAjdMAeEiuIiE3', created_at=1712085355, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::99eQBZYV', finished_at=1712087359, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-zuryypGfetQ6X0k1X2M2uw47', result_files=['file-7jEQtzqoDT7dWgHjRyz4o5br'], status='succeeded', trained_tokens=155439, training_file='file-UQTKBZG6KKPkE4afW6MvCY5m', validation_file=None, user_provided_suffix=None), FineTuningJob(id='ftjob-SBxcyTVGZ3T9WjXJnuejAvdP', created_at=1711839836, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::98cSJKqR', finished_at=1711841474, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', orga

In [31]:

def chatgpt_response(prompt, data=None, model="ft:gpt-3.5-turbo-0125:personal::99eQBZYV"):
    messages = [
        {"role": "system", "content": "You are a helpful and informative chatbot for the first aid assistance."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content.strip()


In [32]:
prompt = "My friend is having fever and headache?"
response = chatgpt_response(prompt, data)
print(response)


I'm sorry to hear about your friend's symptoms. Fever and headache can be caused by various conditions, including infections like the flu or COVID-19, dehydration, sinus issues, or other underlying health issues. Here are some general first aid steps to help your friend feel more comfortable:

1. **Encourage Rest**: Suggest that your friend rests in a comfortable and cool environment to help their body fight off the infection or trigger of the symptoms.

2. **Stay Hydrated**: Encourage them to drink plenty of fluids, such as water, herbal tea, or clear broths, to stay hydrated and help regulate body temperature.

3. **Manage Fever**: You can give them over-the-counter fever-reducing medications like acetaminophen (Tylenol) or ibuprofen (Advil, Motrin) following the recommended dosage instructions.

4. **Cool Compress**: A cool damp cloth on the forehead or back of the neck may help alleviate headache and provide some relief from fever.

5. **Monitor Symptoms**: Keep an eye on your frie

In [34]:
prompt = "Hello I am First Aid GPT here to help you. How are you feeling? "
def tune_response(input, previous):
    previous = previous or []
    s = list(sum(previous, ()))
    s.append(input)
    inp = ' '.join(s)
    output = chatgpt_response(inp)
    previous.append((input, output))
    return previous, previous


block = gr.Blocks(title="First Aid GPT")


with block:
    gr.Markdown("""<h1><center> First Aid GPT</center></h1>
    """)
    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder=prompt)
    state = gr.State()
    submit = gr.Button("SEND")

    submit.click(tune_response, inputs=[message, state], outputs=[chatbot, state])

block.launch(debug = True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3c242e354f902d0c2b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3c242e354f902d0c2b.gradio.live
